In [ ]:
import os 
import getopt
import sys

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.regularizers import l1

np.random.seed(1337) # for reproducibility
vocab = ["A", "G", "C", "T"]
indices = tf.range(len(vocab), dtype = tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
table = tf.lookup.StaticVocabularyTable(table_init, 1)
defs = [0.] * 1 + [tf.constant([], dtype = "string")]

# Nadav dataset

def data_reader(file, batch_size=100, n_parse_threads = 4):
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset=dataset.map(preprocess, num_parallel_calls = n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    fields = tf.io.decode_csv(record, record_defaults=defs)
    chars = tf.strings.bytes_split(fields[1])
    chars_indeces = table.lookup(chars)
    X = tf.one_hot(chars_indeces, depth = len(vocab))
    Y = fields[0]
    return X,Y


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data in three partitions
file="/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3.csv"
whole_data = pd.read_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3.csv")

df_train, df_test = train_test_split(whole_data, test_size=0.2, random_state=42)
df_validation, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

df_train.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3_train.csv", index=False)
df_test.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3_test.csv", index=False)
df_validation.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3_validation.csv", index=False)


In [ ]:
input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3_train.csv"
input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3_validation.csv"
input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3_test.csv"

corr_list = []
df_repetition = pd.DataFrame()

for i in range(0,10):
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(file):
        input_shape = element[0].shape

    inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
    layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
    layer = Dropout(0.3)(layer)
    layer = BatchNormalization()(layer)
    layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
    layer = BatchNormalization()(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
    layer = Dropout(0.3)(layer)
    layer = Flatten()(layer)
    layer = Dense(300, activation='sigmoid')(layer)  # 300
    layer = Dropout(0.3)(layer)
    layer = Dense(200, activation='sigmoid')(layer)  # 300
    predictions = Dense(1, activation='linear')(layer)

    model = Model(inputs=inputs, outputs=predictions)
    model.summary()

    model.compile(optimizer="adam",
                loss="mean_squared_error",
                metrics=["mse", "mae", "mape"],
                )

    history=model.fit(data_reader(input_path_train, batch_size=100),
                            epochs=20,
                            validation_data=data_reader(file,batch_size=100),
                            callbacks=None,
                            verbose=1)

    predicted = model.predict(data_reader(input_path_test,
                                                batch_size=100))

    test_data = data_reader(input_path_test,batch_size=100)
    test_tensor = X = np.empty(shape=[0,1])
    for batch in test_data:
        test_tensor = np.append(test_tensor, batch[1])

    import math
    def pearson_correlation(x, y):
        n = len(x)
        # Calculate the mean of x and y
        mean_x = sum(x) / n
        mean_y = sum(y) / n
        
        # Calculate the numerator and denominators of the correlation coefficient
        numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
        denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
        denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
        
        # Calculate the correlation coefficient
        correlation = numerator / (denominator_x * denominator_y)
        return correlation
        
    corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
    corr_list.append(corr_coefficient)
    df_repetition[str(i)] =predicted.flatten()

In [ ]:
df_copy = df_repetition
df_repetition["col_mean"] = df_repetition.mean(axis = 1)
print(pearson_correlation(df_repetition.col_mean, test_tensor))
print(df_repetition.corr().abs())
df_repetition


In [ ]:

corr_list

In [ ]:
#df_test["prediction"] = predicted
#df_test.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3_test_predicte.csv", index=False)


In [ ]:
from sklearn.model_selection import train_test_split
# Split the data in three partitions
file="/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ.csv"
whole_data = pd.read_csv(file)

df_train, df_test = train_test_split(whole_data, test_size=0.2, random_state=42)
df_validation, df_test = train_test_split(df_test, test_size=0.5, random_state=42)

df_train[["meanVal", "sequence"]].to_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_train.csv", index=False)
df_test[["meanVal", "sequence"]].to_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_test.csv", index=False)
df_validation[["meanVal", "sequence"]].to_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_validation.csv", index=False)

In [ ]:

input_path_train = "/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_train.csv"
input_path_test = "/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_test.csv"
input_path_valid = "/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_validation.csv"

# Get first item of the dataset to get the shape of the input data
for element in data_reader(input_path_train):
    input_shape = element[0].shape

inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
layer = Dropout(0.5)(layer)
layer = BatchNormalization()(layer)
layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
layer = Dropout(0.5)(layer)
layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
layer = BatchNormalization()(layer)
layer = Dropout(0.5)(layer)
layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
layer = Dropout(0.5)(layer)
layer = Flatten()(layer)
layer = Dense(300, activation='sigmoid')(layer)  # 300
layer = Dropout(0.5)(layer)
layer = Dense(200, activation='sigmoid')(layer)  # 300
predictions = Dense(1, activation='linear')(layer)

model = Model(inputs=inputs, outputs=predictions)
model.summary()

model.compile(optimizer="adam",
              loss="mean_squared_error",
              metrics=["mse", "mae", "mape"],
              )

history=model.fit(data_reader(input_path_train, batch_size=100),
                        epochs=15,
                        validation_data=data_reader(input_path_valid,batch_size=100),
                        callbacks=None,
                        verbose=1)

predicted = model.predict(data_reader(input_path_test,
                                            batch_size=100))

test_data = data_reader(input_path_test,batch_size=100)
test_tensor = X = np.empty(shape=[0,1])
for batch in test_data:
    test_tensor = np.append(test_tensor, batch[1])

import math
def pearson_correlation(x, y):
    n = len(x)
    # Calculate the mean of x and y
    mean_x = sum(x) / n
    mean_y = sum(y) / n
    
    # Calculate the numerator and denominators of the correlation coefficient
    numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
    denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
    denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
    
    # Calculate the correlation coefficient
    correlation = numerator / (denominator_x * denominator_y)
    return correlation
    
corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)

In [ ]:
print("Correlation: "+str(corr_coefficient))
print("Test Data : \n"+str(test_tensor[:10]))
print("Predictions :\n" +str(predicted[:10]))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()
    
create_plots(history)

In [ ]:
#Read Data
import shap
_dataset = data_reader("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_2col_test.csv", batch_size=10000)
shuffled_dataset = _dataset.shuffle(5000)
iterator = shuffled_dataset.as_numpy_iterator()
random_sample = next(iterator)
X = random_sample[0]

rn=np.random.choice([X,  X[:,::-1,::-1]][0].shape[0], 1000, replace=False)
explainer = shap.DeepExplainer((model.inputs, model.layers[-1].output), X[rn])
explainer.shap_values(X[0:10], ranked_outputs=1)

#shap_values_top, indexes = explainer.shap_values(X, ranked_outputs=1)

In [ ]:
#Read Data
import shap
_dataset = data_reader("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ_2col_test.csv", batch_size=10000)
shuffled_dataset = _dataset.shuffle(5000)
iterator = shuffled_dataset.as_numpy_iterator()
random_sample = next(iterator)
X = random_sample[0]

In [ ]:
print(X.shape)
rn=np.random.choice([X,  X[:,::-1,::-1]][0].shape[0], 1000, replace=False)
X[rn].shape

In [ ]:
rn=np.random.choice([X,  X[:,::-1,::-1]][0].shape[0], 200, replace=False)
rn

In [ ]:
def one_hot_to_dna_sequence(one_hot_sequence, vocab):
    """
    Convert a one-hot-encoded DNA sequence back to a DNA sequence using the provided vocabulary.
    Args:
    - one_hot_sequence: A tensor containing the one-hot-encoded DNA sequence.
    - vocab: A list of vocabulary characters (e.g., ["A", "G", "C", "T"]).

    Returns:
    - dna_sequence: A tensor containing the decoded DNA sequence.
    """
    # Get the index of the maximum value along the one-hot encoding axis
    decoded_indices = tf.argmax(one_hot_sequence, axis=-1)

    # Use the indices to map back to DNA characters
    dna_sequence = tf.gather(vocab, decoded_indices)

    return dna_sequence

In [ ]:
whole_data

In [ ]:
# CROSS VALIDATION
from sklearn.model_selection import train_test_split, KFold

# Split the data in three partitions
file="/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3.csv"
whole_data = pd.read_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/LibA_wide_pivot_state3.csv")

kf = KFold(n_splits = 10, shuffle = True, random_state = 2008)
result = next(kf.split(whole_data), None)

o=1
for i in kf.split(whole_data):
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    train, validation = train_test_split(whole_data, test_size=0.10, random_state=42)
    
    train.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(o)+"_LibA_wide_pivot_state3_train.csv", index=False)
    test.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(o)+"_LibA_wide_pivot_state3_test.csv", index=False)
    validation.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(o)+"_LibA_wide_pivot_state3_validation.csv", index=False)
    o+=1
        

In [ ]:
df_test_overall  = pd.DataFrame(columns=['State_3E', "seq", "prediction"])
print(df_test_overall)
corr_list = []

for i in range(1,11):
    
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    df_test = pd.read_csv(input_path_test)

    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape

    inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
    layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
    layer = Dropout(0.3)(layer)
    layer = BatchNormalization()(layer)
    layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
    layer = BatchNormalization()(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
    layer = Dropout(0.3)(layer)
    layer = Flatten()(layer)
    layer = Dense(300, activation='sigmoid')(layer)  # 300
    layer = Dropout(0.3)(layer)
    layer = Dense(200, activation='sigmoid')(layer)  # 300
    predictions = Dense(1, activation='linear')(layer)

    model = Model(inputs=inputs, outputs=predictions)
    model.summary()

    model.compile(optimizer="adam",
                loss="mean_squared_error",
                metrics=["mse", "mae", "mape"],
                )

    history=model.fit(data_reader(input_path_train, batch_size=100),
                            epochs=20,
                            validation_data=data_reader(input_path_valid,batch_size=100),
                            callbacks=None,
                            verbose=1)

    predicted = model.predict(data_reader(input_path_test,
                                                batch_size=100))

    test_data = data_reader(input_path_test,batch_size=100)
    test_tensor = X = np.empty(shape=[0,1])
    for batch in test_data:
        test_tensor = np.append(test_tensor, batch[1])

    df_test["prediction"] = predicted
    df_test["fold"] = str(i)
    df_test_overall = df_test_overall.append(df_test, ignore_index=True)

    
    def pearson_correlation(x, y):
        n = len(x)
        # Calculate the mean of x and y
        mean_x = sum(x) / n
        mean_y = sum(y) / n
        
        # Calculate the numerator and denominators of the correlation coefficient
        numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
        denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
        denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
        
        # Calculate the correlation coefficient
        correlation = numerator / (denominator_x * denominator_y)
        return correlation
        
    corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
    corr_list.append(corr_coefficient)

df_test_overall.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()
    
create_plots(history)

In [ ]:
df_test_overall.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)
df_test_overall

In [ ]:
corr_list

In [ ]:
df_test_overall  = pd.DataFrame(columns=['State_3E', "seq", "prediction"])

    
input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV1_LibA_wide_pivot_state3_train.csv"
input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV1_LibA_wide_pivot_state3_validation.csv"
input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV1_LibA_wide_pivot_state3_test.csv"
df_test = pd.read_csv(input_path_test)

# Get first item of the dataset to get the shape of the input data
for element in data_reader(input_path_train):
    input_shape = element[0].shape

inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
layer = Dropout(0.3)(layer)
layer = BatchNormalization()(layer)
layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
layer = Dropout(0.3)(layer)
layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)
layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
layer = Dropout(0.3)(layer)
layer = Flatten()(layer)
layer = Dense(300, activation='sigmoid')(layer)  # 300
layer = Dropout(0.3)(layer)
layer = Dense(200, activation='sigmoid')(layer)  # 300
predictions = Dense(1, activation='linear')(layer)

model = Model(inputs=inputs, outputs=predictions)
model.summary()

model.compile(optimizer="adam",
            loss="mean_squared_error",
            metrics=["mse", "mae", "mape"],
            )

history=model.fit(data_reader(input_path_train, batch_size=100),
                        epochs=20,
                        validation_data=data_reader(input_path_valid,batch_size=100),
                        callbacks=None,
                        verbose=1)

predicted = model.predict(data_reader(input_path_test,
                                            batch_size=100))

test_data = data_reader(input_path_test,batch_size=100)
test_tensor = X = np.empty(shape=[0,1])
for batch in test_data:
    test_tensor = np.append(test_tensor, batch[1])

df_test["prediction"] = predicted
df_test_overall = df_test_overall.append(df_test, ignore_index=True)


def pearson_correlation(x, y):
    n = len(x)
    # Calculate the mean of x and y
    mean_x = sum(x) / n
    mean_y = sum(y) / n
    
    # Calculate the numerator and denominators of the correlation coefficient
    numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
    denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
    denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
    
    # Calculate the correlation coefficient
    correlation = numerator / (denominator_x * denominator_y)
    return correlation
    
corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)


In [ ]:
df_test

In [ ]:
df_test

In [ ]:
# CROSS VALIDATION
from sklearn.model_selection import train_test_split, KFold

# Split the data in three partitions
whole_data=pd.read_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/mean_with_sequence_ENCFF616IAQ.csv")

kf = KFold(n_splits = 10, shuffle = True, random_state = 2008)
result = next(kf.split(whole_data), None)

o=1
for i in kf.split(whole_data):
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    train, validation = train_test_split(whole_data, test_size=0.10, random_state=42)
    
    train[["meanVal", "sequence"]].to_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/10fold_cv/CV"+str(o)+"_mean_with_sequence_ENCFF616IAQ_train.csv", index=False)
    test[["meanVal", "sequence"]].to_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/10fold_cv/CV"+str(o)+"_mean_with_sequence_ENCFF616IAQ_test.csv", index=False)
    validation[["meanVal", "sequence"]].to_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/10fold_cv/CV"+str(o)+"_mean_with_sequence_ENCFF616IAQ_validation.csv", index=False)
    o+=1
        

In [ ]:
df_test_overall  = pd.DataFrame(columns=["meanVal", "Sequence"])
print(df_test_overall)
corr_list = []

for i in range(1,11):
    
    input_path_train = "/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/10fold_cv/CV"+str(i)+"_mean_with_sequence_ENCFF616IAQ_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/10fold_cv/CV"+str(i)+"_mean_with_sequence_ENCFF616IAQ_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/10fold_cv/CV"+str(i)+"_mean_with_sequence_ENCFF616IAQ_test.csv"
   
    df_test = pd.read_csv(input_path_test)
    corr_per_iteration = []
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
        
    for iteration in range(1,11):
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.3)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.3)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.3)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(1, activation='linear')(layer)

        model = Model(inputs=inputs, outputs=predictions)
        model.summary()

        model.compile(optimizer="adam",
                    loss="mean_squared_error",
                    metrics=["mse", "mae", "mape"],
                    )

        history=model.fit(data_reader(input_path_train, batch_size=1024),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=100),
                                callbacks=None,
                                verbose=1)

        predicted = model.predict(data_reader(input_path_test,
                                                    batch_size=100))

        test_data = data_reader(input_path_test,batch_size=100)
        test_tensor = X = np.empty(shape=[0,1])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1])

        df_test["prediction_iteration_"+str(iteration)] = predicted
        df_test["fold"] = str(i)
        df_test_overall = df_test_overall.append(df_test, ignore_index=True)
        
        def pearson_correlation(x, y):
            n = len(x)
            # Calculate the mean of x and y
            mean_x = sum(x) / n
            mean_y = sum(y) / n
            
            # Calculate the numerator and denominators of the correlation coefficient
            numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
            denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
            denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
            
            # Calculate the correlation coefficient
            correlation = numerator / (denominator_x * denominator_y)
            return correlation
                
        corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
        corr_per_iteration.append(corr_coefficient)
        
    corr_ensemble = np.mean(corr_per_iteration)
    corr_list.append(corr_ensemble)

df_test_overall.to_csv("/home/felix/cluster/fpacheco/Data/Nadav_lab/K562/10fold_cv/mean_with_sequence_ENCFF616IAQ_test_predicted_cv10fold.csv", index=False)

In [ ]:
df_test_overall  = pd.DataFrame(columns=["meanVal", "Sequence"])
corr_list = []

for i in range(1,11):
    
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
   
    df_test = pd.read_csv(input_path_test)
    corr_per_iteration = []
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
        
    for iteration in range(1,11):
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.3)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.3)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.3)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(1, activation='linear')(layer)

        model = Model(inputs=inputs, outputs=predictions)
        model.summary()

        model.compile(optimizer="adam",
                    loss="mean_squared_error",
                    metrics=["mse", "mae", "mape"],
                    )

        history=model.fit(data_reader(input_path_train, batch_size=1024),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=100),
                                callbacks=None,
                                verbose=1)

        predicted = model.predict(data_reader(input_path_test,
                                                    batch_size=100))

        test_data = data_reader(input_path_test,batch_size=100)
        test_tensor = X = np.empty(shape=[0,1])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1])

        df_test["prediction_iteration_"+str(iteration)] = predicted
        df_test["fold"] = str(i)
        df_test_overall = df_test_overall.append(df_test, ignore_index=True)
        
        def pearson_correlation(x, y):
            n = len(x)
            # Calculate the mean of x and y
            mean_x = sum(x) / n
            mean_y = sum(y) / n
            
            # Calculate the numerator and denominators of the correlation coefficient
            numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
            denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
            denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
            
            # Calculate the correlation coefficient
            correlation = numerator / (denominator_x * denominator_y)
            return correlation
                
        corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
        corr_per_iteration.append(corr_coefficient)
        
    corr_ensemble = np.mean(corr_per_iteration)
    corr_list.append(corr_ensemble)

df_test_overall.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold_ensemble.csv", index=False)

In [45]:
df_test_overall  = pd.DataFrame()
corr_list = []

for i in range(1,11):
    
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
   
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    corr_per_iteration = []
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
        
    for iteration in range(1,11):
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.3)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.3)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.3)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(1, activation='linear')(layer)

        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer="adam",
                    loss="mean_squared_error",
                    metrics=["mse", "mae", "mape"],
                    )

        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=100),
                                callbacks=None,
                                verbose=2)

        predicted = model.predict(data_reader(input_path_test,
                                                    batch_size=100))

        test_data = data_reader(input_path_test,batch_size=100)
        test_tensor = X = np.empty(shape=[0,1])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1])

        df_test["prediction_iteration_"+str(iteration)] = predicted
        
                
        corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
        corr_per_iteration.append(corr_coefficient)
    
    df_test_overall = df_test_overall.append(df_test, ignore_index=True)
    print(df_test_overall)
        
    corr_ensemble = np.mean(corr_per_iteration)
    corr_list.append(corr_ensemble)

df_test_overall.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold_ensemble.csv", index=False)

Epoch 1/20
69/69 - 5s - loss: 0.0398 - mse: 0.0398 - mae: 0.1514 - mape: 2692.7610 - val_loss: 0.0215 - val_mse: 0.0215 - val_mae: 0.0987 - val_mape: 307.8459 - 5s/epoch - 76ms/step
Epoch 2/20
69/69 - 1s - loss: 0.0177 - mse: 0.0177 - mae: 0.1017 - mape: 13671.0537 - val_loss: 0.0216 - val_mse: 0.0216 - val_mae: 0.0988 - val_mape: 284.9145 - 1s/epoch - 21ms/step
Epoch 3/20
69/69 - 1s - loss: 0.0128 - mse: 0.0128 - mae: 0.0863 - mape: 10752.0742 - val_loss: 0.0209 - val_mse: 0.0209 - val_mae: 0.0998 - val_mape: 488.3447 - 1s/epoch - 21ms/step
Epoch 4/20
69/69 - 1s - loss: 0.0111 - mse: 0.0111 - mae: 0.0803 - mape: 3503.9500 - val_loss: 0.0208 - val_mse: 0.0208 - val_mae: 0.1014 - val_mape: 577.5631 - 1s/epoch - 21ms/step
Epoch 5/20
69/69 - 1s - loss: 0.0104 - mse: 0.0104 - mae: 0.0787 - mape: 2690.9761 - val_loss: 0.0360 - val_mse: 0.0360 - val_mae: 0.1620 - val_mape: 1716.6106 - 1s/epoch - 20ms/step
Epoch 6/20
69/69 - 1s - loss: 0.0093 - mse: 0.0093 - mae: 0.0738 - mape: 2578.6809 - va

2023-10-19 16:56:36.558948: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 679617733643491205


Epoch 1/20
69/69 - 4s - loss: 0.0410 - mse: 0.0410 - mae: 0.1534 - mape: 5896.8110 - val_loss: 0.0210 - val_mse: 0.0210 - val_mae: 0.1034 - val_mape: 661.4275 - 4s/epoch - 62ms/step
Epoch 2/20


2023-10-19 16:56:41.294188: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 679617733643491205


69/69 - 1s - loss: 0.0178 - mse: 0.0178 - mae: 0.1025 - mape: 3386.1479 - val_loss: 0.0208 - val_mse: 0.0208 - val_mae: 0.0998 - val_mape: 492.1875 - 1s/epoch - 20ms/step
Epoch 3/20
69/69 - 1s - loss: 0.0130 - mse: 0.0130 - mae: 0.0868 - mape: 7463.1724 - val_loss: 0.0212 - val_mse: 0.0212 - val_mae: 0.1050 - val_mape: 721.2650 - 1s/epoch - 20ms/step
Epoch 4/20
69/69 - 1s - loss: 0.0115 - mse: 0.0115 - mae: 0.0813 - mape: 5243.7593 - val_loss: 0.0217 - val_mse: 0.0217 - val_mae: 0.0989 - val_mape: 276.2850 - 1s/epoch - 20ms/step
Epoch 5/20
69/69 - 1s - loss: 0.0109 - mse: 0.0109 - mae: 0.0804 - mape: 14599.3779 - val_loss: 0.0208 - val_mse: 0.0208 - val_mae: 0.1018 - val_mape: 597.8484 - 1s/epoch - 20ms/step
Epoch 6/20
69/69 - 1s - loss: 0.0090 - mse: 0.0090 - mae: 0.0723 - mape: 4757.0366 - val_loss: 0.0216 - val_mse: 0.0216 - val_mae: 0.1080 - val_mape: 814.9133 - 1s/epoch - 21ms/step
Epoch 7/20
69/69 - 1s - loss: 0.0074 - mse: 0.0074 - mae: 0.0664 - mape: 1153.3442 - val_loss: 0.022

...

2023-10-19 17:49:19.242010: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7627355495656467413
2023-10-19 17:49:19.242099: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 679617733643491205


69/69 - 4s - loss: 0.0399 - mse: 0.0399 - mae: 0.1510 - mape: 1062.6202 - val_loss: 0.0172 - val_mse: 0.0172 - val_mae: 0.0991 - val_mape: 93570.9297 - 4s/epoch - 61ms/step
Epoch 2/20
69/69 - 1s - loss: 0.0171 - mse: 0.0171 - mae: 0.0994 - mape: 629.6453 - val_loss: 0.0169 - val_mse: 0.0169 - val_mae: 0.0973 - val_mape: 85429.0547 - 1s/epoch - 20ms/step
Epoch 3/20
69/69 - 1s - loss: 0.0124 - mse: 0.0124 - mae: 0.0841 - mape: 616.2618 - val_loss: 0.0169 - val_mse: 0.0169 - val_mae: 0.0969 - val_mape: 83839.6875 - 1s/epoch - 20ms/step
Epoch 4/20
69/69 - 1s - loss: 0.0109 - mse: 0.0109 - mae: 0.0787 - mape: 550.8186 - val_loss: 0.0169 - val_mse: 0.0169 - val_mae: 0.0972 - val_mape: 85216.7344 - 1s/epoch - 20ms/step
Epoch 5/20
69/69 - 1s - loss: 0.0096 - mse: 0.0096 - mae: 0.0747 - mape: 520.0292 - val_loss: 0.0173 - val_mse: 0.0173 - val_mae: 0.0996 - val_mape: 95501.8984 - 1s/epoch - 19ms/step
Epoch 6/20
69/69 - 1s - loss: 0.0092 - mse: 0.0092 - mae: 0.0746 - mape: 504.4758 - val_loss: 0

2023-10-19 17:49:46.639919: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 679617733643491205


Epoch 1/20
69/69 - 4s - loss: 0.0596 - mse: 0.0596 - mae: 0.1789 - mape: 1389.1436 - val_loss: 0.0181 - val_mse: 0.0181 - val_mae: 0.0937 - val_mape: 8299.8008 - 4s/epoch - 61ms/step
Epoch 2/20
69/69 - 1s - loss: 0.0197 - mse: 0.0197 - mae: 0.1070 - mape: 857.3136 - val_loss: 0.0181 - val_mse: 0.0181 - val_mae: 0.0937 - val_mape: 8808.0488 - 1s/epoch - 20ms/step
Epoch 3/20
69/69 - 1s - loss: 0.0136 - mse: 0.0136 - mae: 0.0881 - mape: 855.0131 - val_loss: 0.0179 - val_mse: 0.0179 - val_mae: 0.0932 - val_mape: 12282.7344 - 1s/epoch - 20ms/step
Epoch 4/20
69/69 - 1s - loss: 0.0115 - mse: 0.0115 - mae: 0.0817 - mape: 512.6207 - val_loss: 0.0184 - val_mse: 0.0184 - val_mae: 0.0944 - val_mape: 3947.4163 - 1s/epoch - 20ms/step
Epoch 5/20
69/69 - 1s - loss: 0.0107 - mse: 0.0107 - mae: 0.0792 - mape: 604.0748 - val_loss: 0.0197 - val_mse: 0.0197 - val_mae: 0.0977 - val_mape: 14108.2383 - 1s/epoch - 20ms/step
Epoch 6/20
69/69 - 1s - loss: 0.0094 - mse: 0.0094 - mae: 0.0748 - mape: 615.1165 - val

/tmp/ipykernel_921242/3709728607.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_overall = df_test_overall.append(df_test, ignore_index=True)


In [47]:

df_test_overall
prediction_columns = [col for col in df.columns if col.startswith("prediction_iteration")]
df_test_overall["average_pred"]=df_test_overall[prediction_columns].mean(axis=1)
df_test_overall
df_test_overall.groupby(['fold'])['average_pred', "State_3E"].mean()
df_test_overall.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold_ensemble.csv", index=False)

/tmp/ipykernel_921242/4174027196.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_test_overall.groupby(['fold'])['average_pred', "State_3E"].mean()


In [48]:
df_test_overall

,State_3E,seq,fold,prediction_iteration_1,prediction_iteration_2,prediction_iteration_3,prediction_iteration_4,prediction_iteration_5,prediction_iteration_6,prediction_iteration_7,prediction_iteration_8,prediction_iteration_9,prediction_iteration_10,average_pred
0,-0.007714,AGGACCGGATCAACTAAACAACTCAAACAAGGGCTAATATAACCCA...,1,0.018599,-0.052721,-0.024224,0.016590,0.054284,0.048035,0.010526,0.033011,-0.017738,0.019669,0.010603
1,0.137953,AGGACCGGATCAACTAAACACTAGTCATACTTAAAAATTGCAAGGA...,1,0.013695,-0.029601,-0.038640,-0.037456,0.005326,0.028433,-0.023462,-0.015440,0.007778,0.023349,-0.006602
2,-0.048706,AGGACCGGATCAACTAAACAGGTTCTGACGTATGCTCCTCTATGGA...,1,0.003729,-0.042231,-0.011073,-0.045632,0.021966,-0.005518,-0.027231,-0.039933,-0.059304,-0.038919,-0.024415
3,-0.052804,AGGACCGGATCAACTAAACCCGAGCCTGCCTAGCCCTAGCTTCTCT...,1,0.000083,-0.057219,-0.004733,0.011383,0.036322,0.061904,-0.007352,0.031200,0.004303,0.008730,0.008462
4,0.213652,AGGACCGGATCAACTAAACGGAGCAGAGTTAGTGTCAGGTCAAAAA...,1,0.012500,-0.022670,0.009330,0.006334,-0.009329,0.024110,-0.003123,0.039434,-0.008337,0.004606,0.005286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8473,0.167100,AGGACCGGATCAACTTTTCCGCCTTTTATTATCAGGACTTCACGGG...,10,0.135312,0.176505,0.238588,0.211985,0.281228,0.126949,0.303350,0.165585,0.149945,0.202226,0.199167
8474,0.099489,AGGACCGGATCAACTTTTCGCTCATTAGTACAGGGTATAACGGAAG...,10,0.006182,0.039173,0.113711,0.118104,0.038279,0.035566,0.078208,0.050476,0.003096,0.029139,0.051193
8475,-0.046939,AGGACCGGATCAACTTTTGGTCGGTTGACGGTCGCCTTGATTATTC...,10,0.021159,0.058779,0.121185,0.126376,0.119010,0.036928,0.064429,0.064415,0.010759,0.048589,0.067163
8476,0.093662,AGGACCGGATCAACTTTTTTATCTGGTTATCATTCTAGTCTAGTGC...,10,0.031899,0.067601,0.131569,0.141819,0.099618,0.034195,0.211555,0.189405,0.031991,0.112303,0.105195
